In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import tiktoken
import json
import pandas as pd
from dotenv import load_dotenv

In [ ]:
load_dotenv('.env')
AZURE_OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT=os.getenv('AZURE_OPENAI_ENDPOINT')
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [9]:
summaries_filenames = os.listdir('./local_tests_data/relevant_articles_summaries')
n_summaries_to_embed = len(summaries_filenames)
print(f"Number of summaries to embed: {n_summaries_to_embed}")

Number of summaries to embed: 41


In [14]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

In [15]:
chunked_filenames = [[]]

running_token_count = 0

for filename in summaries_filenames:
    with open(f'./local_tests_data/relevant_articles_summaries/{filename}', 'r') as f:
        content = json.load(f)
        summary = content['content']

        tokens = encoding.encode(summary)
        token_count = len(tokens)

        running_token_count += token_count
        if running_token_count > 8000: # true limit is 8192, but we leave some space for safety
            chunked_filenames.append([])
            running_token_count = 0
        
        chunked_filenames[-1].append(filename)

In [24]:
all_embeddings = []

for i,chunk in enumerate(chunked_filenames):
    print(f"embedding chunk {i} of {len(chunked_filenames)} with {len(chunk)} summaries")
    chunk_article_ids = [filename.split('.')[0] for filename in chunk]
    chunk_summaries = []
    for filename in chunk:
        with open(f'./local_tests_data/relevant_articles_summaries/{filename}', 'r') as f:
            content = json.load(f)
            summary = content['content']
            chunk_summaries.append(summary)

    response = client.embeddings.create(
        input=chunk_summaries,
        model="text-embedding-3-small",
    )

    for article_id, item in zip(chunk_article_ids, response.data):
        all_embeddings.append({
            'article_id': article_id,
            'summary_embedding': item.embedding
        })

embedding chunk 0 of 1 with 41 summaries


In [28]:
pd_embeddings = pd.DataFrame(all_embeddings) 
pd_embeddings.head()

,article_id,summary_embedding
0,business_insider-20250528111956214143,"[0.026244841516017914, 0.02722984366118908, 0...."
1,business_insider-20250528111956214189,"[0.03143119066953659, -0.0455034039914608, 0.0..."
2,business_insider-20250528111956214168,"[0.0054952772334218025, -0.02758956141769886, ..."
3,the_next_web-20250528111956213312,"[0.044054314494132996, -0.026741232722997665, ..."
4,business_insider-20250528111956214162,"[0.002276916755363345, -0.060238227248191833, ..."


In [27]:
pd_embeddings.to_csv('./local_tests_data/relevant_articles_summaries_embeddings/relevant_articles_summaries_embeddings.csv', index=False)

In [31]:
type(all_embeddings[0]['summary_embedding'])

list